In [1]:
# Importing libraries
import pandas as pd    
import numpy as np
import pandas_profiling
import keras
from keras.models import Sequential
from keras.layers import Dense

# Reading data to Dataframes
  
data_train = pd.read_csv('../titanic/train.csv')
data_test = pd.read_csv('../titanic/test.csv')
data_check = pd.read_csv('../titanic/TITANIC_REAL_TEST.csv')

Using TensorFlow backend.


In [2]:
data_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:

#preenchendo valores faltantes com a média
values = {'Age': data_train.Age.mode()[0]}
data_train.fillna(value = values, inplace = True)

values = {'Age': data_test.Age.mode()[0]}
data_test.fillna(value = values, inplace = True)

#mudando o tipo de dado
data_train.Sex = data_train.Sex.astype('category')
data_train.Age = data_train.Age.astype('int32')

data_test.Sex = data_test.Sex.astype('category')
data_test.Age = data_test.Age.astype('int32')

In [4]:
data_train = data_train.drop(['Name','Ticket','Fare','Cabin','Embarked'],axis =1)

data_test = data_test.drop(['Name','Ticket','Fare','Cabin','Embarked'],axis =1)

In [5]:
# create dummies for pitching team, batting team, pitcher id, batter id
#0 para masculino e 1 para feminino
dummies = pd.get_dummies(data_train['Sex'])
data_train['Sex'] = dummies


dummies = pd.get_dummies(data_test['Sex'])
data_test['Sex'] = dummies


In [6]:
data_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,0,22,1,0
1,2,1,1,1,38,1,0
2,3,1,3,1,26,0,0
3,4,1,1,1,35,1,0
4,5,0,3,0,35,0,0


In [7]:
y_train = data_train['Survived'] 
x_test = data_test[['Pclass', 'Sex', 'Age', 'SibSp' , 'Parch'] ]

x_train = data_train[['Pclass', 'Sex', 'Age', 'SibSp' , 'Parch'] ]

In [8]:
x_test

,Pclass,Sex,Age,SibSp,Parch
0,3,0,34,0,0
1,3,1,47,1,0
2,2,0,62,0,0
3,3,0,27,0,0
4,3,1,22,1,1
...,...,...,...,...,...
413,3,0,21,0,0
414,1,1,39,0,0
415,3,0,38,0,0
416,3,0,21,0,0


In [9]:
#criando o modelo
classifier = Sequential()

#Input layer with 5 inputs neurons
classifier.add(Dense(output_dim = 5, init = 'uniform', activation = 'relu', input_dim = 5))
#Hidden layer
classifier.add(Dense(output_dim = 5, init = 'uniform', activation = 'relu'))
#output layer with 1 output neuron which will predict 1 or 0
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))


In [10]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
x_train

,Pclass,Sex,Age,SibSp,Parch
0,3,0,22,1,0
1,1,1,38,1,0
2,3,1,26,0,0
3,1,1,35,1,0
4,3,0,35,0,0
...,...,...,...,...,...
886,2,0,27,0,0
887,1,1,19,0,0
888,3,1,24,1,2
889,1,0,26,0,0


In [12]:
classifier.fit(x_train, y_train, batch_size = 3, nb_epoch = 50)

Epoch 1/50
891/891 [==============================] - 1s 2ms/step - loss: 0.6818 - accuracy: 0.6128
Epoch 2/50
891/891 [==============================] - 1s 1ms/step - loss: 0.6578 - accuracy: 0.6162
Epoch 3/50
891/891 [==============================] - 1s 1ms/step - loss: 0.6404 - accuracy: 0.6162
Epoch 4/50
891/891 [==============================] - 1s 1ms/step - loss: 0.5935 - accuracy: 0.6162
Epoch 5/50
891/891 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.6566
Epoch 6/50
891/891 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7666
Epoch 7/50
891/891 [==============================] - 1s 2ms/step - loss: 0.5243 - accuracy: 0.7890
Epoch 8/50
891/891 [==============================] - 1s 1ms/step - loss: 0.5163 - accuracy: 0.7912
Epoch 9/50
891/891 [==============================] - 1s 2ms/step - loss: 0.4880 - accuracy: 0.8013
Epoch 10/50
891/891 [==============================] - 1s 1ms/step - loss: 0.4658 - accuracy: 0.8025

In [13]:
#getting predictions of test data
prediction = classifier.predict(x_test).tolist()
# list to series
se = pd.Series(prediction)


# creating new column of predictions in data_check dataframe
data_check['check'] = se
data_check['check'] = data_check['check'].str.get(0)

series = []
for val in data_check.check:
    if val >= 0.5:
        series.append(1)
    else:
        series.append(0)
data_check['final'] = series

In [14]:
data_check

,PassengerId,Survived,check,final
0,892,0,0.062530,0
1,893,1,0.411181,0
2,894,0,0.053704,0
3,895,0,0.105596,0
4,896,1,0.482714,0
...,...,...,...,...
413,1305,0,0.161500,0
414,1306,1,0.970194,1
415,1307,0,0.045921,0
416,1308,0,0.161500,0


In [15]:
data_check['Survived']

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    1
Name: Survived, Length: 418, dtype: int64

In [16]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(data_check['Survived'], data_check['final']))

[[221  38]
 [ 57 102]]
